In [37]:
import pandas as pd
import re

# Shows more characters in the output when you print data
pd.options.display.max_colwidth = 100

#def format_data(user_id):
# Reads in the Transaction and Balance Json files from Nordigen, Drops the Transaction ID column since it has
# useless information
def format_data(user_id):
    df = pd.read_json(fr"C:\Users\Jack\Documents\GitHub\NetWorth\Bank Account Transactions\Formatting\User Transaction Data\{user_id}_Transactions.json")

    # Pending transactions does not sync with the balance in Nordigen, as a result we are adding the pending transactions to
    # our booked transactions to compensate and show the proper balance.
    df_transactions = pd.DataFrame(df['transactions']['booked'])
    df_transactions_pending = pd.DataFrame(df['transactions']['pending'])

    df_transactions = pd.concat([df_transactions, df_transactions_pending])
    df_transactions["bookingDate"] = pd.to_datetime(df_transactions["bookingDate"])
    df_transactions.reset_index(inplace=True, drop=True)

    df_transactionamount = pd.json_normalize(df_transactions['transactionAmount'])
    df_transactions = pd.concat([df_transactions, df_transactionamount], axis=1).drop('transactionAmount', axis = 1)
    df_transactions.drop(columns=['transactionId'], inplace=True)
    df_transactions = df_transactions.sort_values(by='bookingDate', ascending=False)
    df_transactions.reset_index(inplace=True, drop=True)
    print(df_transactions.dtypes)
    print(df_transactions)
    #%%

    df = pd.read_json(fr"C:\Users\Jack\Documents\GitHub\NetWorth\Bank Account Transactions\Formatting\User Transaction Data\Testid_00000009_Balances.json")
    # Since we will use booked transactions from the "Transactions" data, we only want to see the forwardavailable
    # balance from the "Balance" data
    df_balance = pd.DataFrame(df['balances'][1])

    #%%

    # This creates a balance column using the current balance in the "balance" data taken from the Nordigen API
    current_balance = []
    value = float(df_balance['balanceAmount'].loc['amount'])
    for balance in df_transactions['amount']:
        current_balance.append(value)
        value -= float(balance)
    df_transactions['balance'] = current_balance

    # This removed all the unexpected whitespace, that is longer than expected, reducing them to a specified size, '  ',
    # this space is then replaced by ',' so that it can be separated by the comma later when creating new columns.
    finished = False
    while not finished:
        counter = 0
        for index, description in enumerate(df_transactions['remittanceInformationUnstructured']):
            Tokens = re.findall('\s+', description)
            for token in Tokens:
                if len(token) > len('  '):
                    df_transactions.loc[index,'remittanceInformationUnstructured'] = df_transactions.loc[index,'remittanceInformationUnstructured'].replace(' '*len(token),'  ')
                    counter += 1
        if counter == 0:
            finished = True
            print("Finished!")
        else:
            print("Needs another go!")
    df_transactions['remittanceInformationUnstructured'] = df_transactions['remittanceInformationUnstructured'].str.replace('  ',',').str.strip()


    # Note: need to split the transaction descriptions into separate columns.
    #Description_Cols = df_transactions["remittanceInformationUnstructured"].str.split(",", expand = True)
    #print(Description_Cols)

    df = pd.read_csv('Transaction Explanations.txt',sep = ';')
    df_transactions.rename(columns = {'proprietaryBankTransactionCode':'Transaction Code'}, inplace = True)
    df_transactions = df_transactions.merge(df, on ='Transaction Code', how='left')
    print(df_transactions)
format_data("Testid_00000009")

bookingDate                          datetime64[ns]
proprietaryBankTransactionCode               object
remittanceInformationUnstructured            object
amount                                       object
currency                                     object
dtype: object
   bookingDate proprietaryBankTransactionCode  \
0   2021-11-30                            BAC   
1   2021-11-29                            POS   
2   2021-11-29                            POS   
3   2021-11-19                            POS   
4   2021-11-19                            POS   
5   2021-11-17                            POS   
6   2021-11-15                            POS   
7   2021-11-15                            POS   
8   2021-11-15                            POS   
9   2021-11-15                            DPC   
10  2021-11-15                            POS   
11  2021-11-12                            POS   
12  2021-11-12                            POS   
13  2021-11-12                          